In [2]:
# === Imports and configuration ===
import os
import glob
import time
import pandas as pd
import numpy as np
import requests

# Global variables
START_DATE = '2024-01-01'
END_DATE = '2024-12-31'

# Csv files paths
INPUT_DIR = "2024_capital_cities_data"
INPUT_PATTERN = os.path.join(INPUT_DIR, "2024_data_*.csv")

# output
OUTPUT_CSV = "solar_weather_all_cities.csv"

# API
API_SLEEP_S = 5            # segundos de pausa entre llamadas para no saturar
USE_METEO_CACHE = True     # cachear meteo por ciudad/año
CACHE_DIR = "meteo_cache"
os.makedirs(CACHE_DIR, exist_ok=True)

In [3]:
# === City coordinates ===
d = {
    'city': [
        'Madrid', 'Paris', 'Berlin', 'Rome', 'Vienna', 'Brussels', 'Sofia',
        'Zagreb', 'Nicosia', 'Prague', 'Copenhagen', 'Tallinn', 'Helsinki',
        'Athens', 'Budapest', 'Dublin', 'Riga', 'Vilnius', 'Luxembourg',
        'Valletta', 'Amsterdam', 'Warsaw', 'Lisbon', 'Bucharest', 'Bratislava',
        'Ljubljana', 'Stockholm', 'London'
    ],
    'latitude': [
        40.4165, 48.85341, 52.52281, 41.89306, 48.2082, 50.8503, 42.6977,
        45.815, 35.1856, 50.0755, 55.6761, 59.437, 60.1699, 37.9838, 47.4979,
        53.3498, 56.9496, 54.6872, 49.6116, 35.8989, 52.3676, 52.2297,
        38.7223, 44.4268, 48.1486, 46.0569, 59.3293, 51.5074
    ],
    'longitude': [
        -3.70256, 2.3488, 13.408176, 12.48278, 16.3738, 4.3517, 23.3219,
        15.9819, 33.3823, 14.4378, 12.5683, 24.7536, 24.9384, 23.7275,
        19.0402, -6.2603, 24.1052, 25.2797, 6.1319, 14.5146, 4.9041,
        21.0122, -9.1393, 26.1025, 17.1077, 14.5058, 18.0686, -0.1278
    ]
}
coord_df = pd.DataFrame(d).set_index("city")
coord_df.head()

,latitude,longitude
city,,
Madrid,40.41650,-3.702560
Paris,48.85341,2.348800
Berlin,52.52281,13.408176
Rome,41.89306,12.482780
Vienna,48.20820,16.373800


In [4]:
# === Multipurpose functions ===
def city_from_filename(path):
    """Extract city exactly as appeas in the file"""
    name = os.path.basename(path)
    return name.replace("2024_data_", "").replace(".csv", "")

def output_has_city(path, city):
    """Checks if city is already in master csv to continue without duplicates"""
    if not os.path.exists(path):
        return False
    for chunk in pd.read_csv(path, chunksize=200_000):
        if "city" in chunk.columns and (chunk["city"] == city).any():
            return True
    return False

def append_csv(df, path):
    header = not os.path.exists(path)
    df.to_csv(path, mode="a", header=header, index=False)

def read_city_solar_csv(file):
    """Read irradiance csv file of a city and creates 1-min timestamp index."""
    df = pd.read_csv(file, sep=';')
    num_rows = len(df)
    times = pd.date_range(START_DATE, freq='1min', periods=num_rows).tz_localize('UTC')
    df = df.set_index(times)
    df.index.names = ['time']
    return df[['GHI', 'BHI', 'DHI']].reset_index()

In [5]:
# === Meteo API ===
def get_city_coords(city):
    if city not in coord_df.index:
        raise ValueError(f"Coordenadas no encontradas para ciudad: {city}")
    row = coord_df.loc[city]
    return float(row["latitude"]), float(row["longitude"])

def minute_index_for_year(start_date, periods):
    return pd.date_range(start_date, freq='1min', periods=periods, tz='UTC')

def fetch_meteo_city(city, start_date, end_date):
    """Descarga meteo para UNA ciudad y la expande a minuto; usa caché parquet."""
    lat, lon = get_city_coords(city)
    year = pd.Timestamp(start_date).year
    cache_path = os.path.join(CACHE_DIR, f"meteo_{city}_{year}.parquet")

    if USE_METEO_CACHE and os.path.exists(cache_path):
        dfh = pd.read_parquet(cache_path)
        # aseguramos que esté en UTC
        if dfh.index.tz is None:
            dfh.index = dfh.index.tz_localize("UTC")
        else:
            dfh.index = dfh.index.tz_convert("UTC")
        return dfh

    variable = "temperature_2m,wind_speed_10m"
    url = (f"https://archive-api.open-meteo.com/v1/era5"
           f"?latitude={lat}&longitude={lon}&hourly={variable}"
           f"&timezone=GMT&start_date={start_date}&end_date={end_date}")
    resp = requests.get(url).json()
    dfh = pd.DataFrame(resp["hourly"])  # hourly
    # Expandir cada registro horario en 60 minutos
    dfh = dfh.loc[dfh.index.repeat(60)].reset_index(drop=True)

    periods = 60 * 24 * (366 if pd.Timestamp(str(year)).is_leap_year else 365)
    idx = minute_index_for_year(start_date, periods)
    dfh["time"] = idx
    dfh["city"] = city
    dfh["lat"] = lat
    dfh["lon"] = lon

    dfh = dfh.set_index("time")
    if dfh.index.tz is None:
        dfh.index = dfh.index.tz_localize("UTC")
    else:
        dfh.index = dfh.index.tz_convert("UTC")

    if USE_METEO_CACHE:
        dfh.to_parquet(cache_path, index=True)
    time.sleep(API_SLEEP_S)
    return dfh

In [6]:
# === City processing ===
def process_one_city(file, output_csv=OUTPUT_CSV):
    city = city_from_filename(file)
    if output_has_city(output_csv, city):
        print(f"⏭️  Saltando {city}: ya en {output_csv}")
        return

    print(f"🔄 Procesando {city}")
    solar_df = read_city_solar_csv(file)
    meteo_df = fetch_meteo_city(city, START_DATE, END_DATE).reset_index()

    merged = solar_df.merge(meteo_df, on="time", how="inner").rename(columns={
        'GHI': 'ghi',
        'BHI': 'dni',     # tu CSV usa BHI -> DNI
        'DHI': 'dhi',
        'wind_speed_10m': 'wind_speed',
        'temperature_2m': 'temp_air'
    })
    # Conversión de unidades
    merged['ghi'] *= 60
    merged['dni'] *= 60
    merged['dhi'] *= 60
    merged['wind_speed'] *= (1000/3600)
    merged['city'] = city

    append_csv(merged, output_csv)
    print(f"✅ {city} añadido ({len(merged):,} filas)")

In [7]:
# === Batch running (memory limit) ===
def run_batch(target_cities=None):
    files = glob.glob(INPUT_PATTERN)
    if target_cities:
        target = set(target_cities)  # ya vienen capitalizadas en tus archivos
        files = [f for f in files if city_from_filename(f) in target]

    if not files:
        print("⚠️ No hay ficheros que procesar")
        return

    for f in sorted(files):
        try:
            process_one_city(f, OUTPUT_CSV)
        except Exception as e:
            print(f"❌ Error en {f}: {e}")

In [10]:
# run_batch()

# Example: batch
run_batch(["Madrid", "Paris", "Berlin", "Rome", "Vienna", "Brussels", "Sofia", 
 "Zagreb", "Nicosia", "Prague", "Copenhagen", "Tallinn", "Helsinki",
 "Athens", "Budapest", "Dublin", "Riga", "Vilnius", "Luxembourg",
 "Valletta", "Amsterdam", "Warsaw", "Lisbon", "Bucharest", "Bratislava",
 "Ljubljana", "Stockholm", "London"])

🔄 Procesando Amsterdam
✅ Amsterdam añadido (527,040 filas)
🔄 Procesando Athens
✅ Athens añadido (527,040 filas)
🔄 Procesando Berlin
✅ Berlin añadido (527,040 filas)
🔄 Procesando Bratislava
✅ Bratislava añadido (527,040 filas)
🔄 Procesando Brussels
✅ Brussels añadido (527,040 filas)
🔄 Procesando Bucharest
✅ Bucharest añadido (527,040 filas)
🔄 Procesando Budapest
✅ Budapest añadido (527,040 filas)
🔄 Procesando Copenhagen
✅ Copenhagen añadido (527,040 filas)
🔄 Procesando Dublin
✅ Dublin añadido (527,040 filas)
🔄 Procesando Helsinki
✅ Helsinki añadido (527,040 filas)
🔄 Procesando Lisbon
✅ Lisbon añadido (527,040 filas)
🔄 Procesando Ljubljana
✅ Ljubljana añadido (527,040 filas)
🔄 Procesando London
✅ London añadido (527,040 filas)
🔄 Procesando Luxembourg
✅ Luxembourg añadido (527,040 filas)
⏭️  Saltando Madrid: ya en solar_weather_all_cities.csv
🔄 Procesando Nicosia
✅ Nicosia añadido (527,040 filas)
🔄 Procesando Paris
✅ Paris añadido (527,040 filas)
🔄 Procesando Prague
✅ Prague añadido (527,

In [11]:
df_out = pd.read_csv(OUTPUT_CSV, nrows=10)
print(df_out.head())

                        time  ghi  dni  dhi  temp_air  wind_speed    city  \
0  2024-01-01 00:00:00+00:00  0.0  0.0  0.0       4.2    1.305556  Madrid   
1  2024-01-01 00:01:00+00:00  0.0  0.0  0.0       4.2    1.305556  Madrid   
2  2024-01-01 00:02:00+00:00  0.0  0.0  0.0       4.2    1.305556  Madrid   
3  2024-01-01 00:03:00+00:00  0.0  0.0  0.0       4.2    1.305556  Madrid   
4  2024-01-01 00:04:00+00:00  0.0  0.0  0.0       4.2    1.305556  Madrid   

       lat      lon  
0  40.4165 -3.70256  
1  40.4165 -3.70256  
2  40.4165 -3.70256  
3  40.4165 -3.70256  
4  40.4165 -3.70256  
